# Arc Results

In [ ]:
%matplotlib inline

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sbn

from smif.data_layer import Results

results = Results({'interface': 'local_csv', 'dir': '../'})
store = results._store

sbn.set_context('talk', rc={"lines.linewidth": 2.5})
sbn.set_palette('Paired')

In [ ]:
lads = gpd.read_file('../data/dimensions/arc_lad_uk16.gpkg')
lads.head(1)

In [ ]:
arc_lads = lads[lads.in_arc == 1]

In [ ]:
arc_lads.to_file('../data/dimensions/lad_uk_2016-12/arc_lads.shp')

In [ ]:
arc_lads = arc_lads.drop(columns=['in_arc']).rename(columns={'name': 'lad_uk_2016'})
arc_lads.head(2)

# View the LADs

In [ ]:
highlight_lads = arc_lads[arc_lads.desc.isin({'Bedford', 'Cambridge', 'Milton Keynes', 'Oxford', 'Cherwell',
                     'Aylesbury Vale', 'Central Bedfordshire', 'Huntingdonshire', 'South Cambridgeshire'})]
highlight_lads

In [ ]:
def highlight(data, id_col='desc'):
    data = data.copy()
    
    def group_hlads(index):
        if index in list(highlight_lads.desc):
            return index
        else:
            return 'Other'
    
    data['summary_lads'] = data[id_col].apply(group_hlads) 
    
    return data
    
def plot_lads(arc_lads, label_col):
    # Plot the LADs
    ax = arc_lads.plot(column=label_col, edgecolor='grey')
    
    # Add the labels
    centroids = [(float(x.x), float(x.y)) for x in list(arc_lads.centroid)]
    for label, coord in zip(arc_lads[label_col].values, centroids):
        if label != 'Other':
            plt.annotate(label, coord, ha='center', va='center',
                         color='black', weight='bold')
    ax.set_axis_off()
    return ax
    
def plot_highlighted_lads(arc_lads):    
    arc_lads = highlight(arc_lads.reset_index())
    
    return plot_lads(arc_lads, 'summary_lads')

In [ ]:
with sbn.plotting_context('paper'):
    sbn.set(rc={'figure.figsize': (11.7, 8.27)})
    sbn.set_palette('Paired')
    ax = plot_highlighted_lads(arc_lads)
    ax.get_figure().savefig("lads_highlighted.png", transparent=True)
    
with sbn.plotting_context('paper'):
    sbn.set(rc={'figure.figsize': (11.7, 8.27)})
    sbn.set_palette('Paired')
    ax = plot_lads(arc_lads, 'desc')
    ax.get_figure().savefig("lads_all.png", transparent=True)

In [ ]:
highlight(arc_lads)

In [ ]:
results.list_model_runs()

In [ ]:
results.available_results('arc_ets__baseline')

In [ ]:
results.list_scenario_outputs('socio-economic')
dwellings = results.read_scenario_data(
    'socio-economic', 'arc_baseline', 'dwellings', [2015, 2020, 2030, 2050])
dwellings.head()

# Plotting Arc scenario data

The outputs of the socio economic scenarios include:

In [ ]:
results.list_scenario_outputs('socio-economic')

In [ ]:
results.available_results('arc_et__baseline')

In [ ]:
def get_scenario_data(output, arc_lads):
    timesteps = [2015, 2020, 2030, 2050]
    scenarios = ['arc_baseline', 'arc_unplanned', 'arc_new-cities', 'arc_expansion']
    
    dfs = []
    for scenario in scenarios:
        df = results.read_scenario_data(
            'socio-economic', 
            scenario, 
            output, 
            timesteps
        )
        df = df[df.lad_uk_2016.isin(arc_lads.lad_uk_2016)] 
        df['scenario'] = scenario.replace("arc_", "")
        
        dfs.append(df)        
    
    data = pd.concat(dfs)     
    data = data.merge(arc_lads, how='left', on='lad_uk_2016')
    data = gpd.GeoDataFrame(data)
    data.crs = arc_lads.crs
    
    return data

In [ ]:
def plot_total_scenario_data(data, y):
    data = data.groupby(
        by=['scenario', 'timestep']
    ).sum().reset_index()
    
    return sbn.relplot(
        data=data, 
        x='timestep', 
        y=y, 
        hue='scenario', 
        kind='line'
    )
    
def plot_scenario_data(data, y):      
    return sbn.relplot(
        data=data, 
        x='timestep', 
        y=y, 
        hue='desc', 
        col='scenario', 
        kind='line',
        col_wrap=2
    )
    
def plot_scenario_data_highlight(data, y):
    data = highlight(data)   
    return sbn.relplot(
        data=data, 
        x='timestep', 
        y=y, 
        hue='summary_lads', 
        col='scenario', 
        kind='line', 
        col_wrap=2
    )

def plot_bar_highlight(data, y):
    return sbn.catplot(
        data=data, 
        x='timestep', 
        y=y, 
        hue='summary_lads', 
        col='scenario', 
        kind='bar', 
        col_wrap=2
    )   

In [ ]:
arc_dwellings = get_scenario_data('dwellings', arc_lads)
arc_floor = get_scenario_data('floor_area', arc_lads)
arc_population = get_scenario_data('population', arc_lads)
arc_jobs = get_scenario_data('employment', arc_lads)
arc_gva = get_scenario_data('gva', arc_lads)

# Population and dwelling density

In [ ]:
arc_dwellings['dwellings_density'] = (arc_dwellings.dwellings / arc_dwellings.area) * 1e3
dense_dw = arc_dwellings[arc_dwellings.dwellings_density > 1]
fig = plot_scenario_data(dense_dw, 'dwellings_density')
fig.savefig("dwellings_density.png")
fig

In [ ]:
arc_population['population_density'] = (arc_population.population / arc_population.area) * 1e3
dense_pop = arc_population[arc_population.population_density > 2]
fig = plot_scenario_data(dense_pop, 'population_density')
fig.savefig("population_density.png")
fig

# Plotting of scenario data

In [ ]:
fig = plot_scenario_data_highlight(arc_dwellings, 'dwellings')
fig.savefig("dwellings_highlight.png")
fig

In [ ]:
fig = plot_scenario_data_highlight(arc_floor[arc_floor.residential_or_non == 'residential'], 'floor_area')
fig.savefig("floor_area_highlight.png")
fig

In [ ]:
fig = plot_scenario_data_highlight(arc_floor[arc_floor.residential_or_non == 'non_residential'], 'floor_area')
fig.savefig("floor_area_nr_highlight.png")
fig

In [ ]:
fig = plot_bar_highlight(arc_dwellings, 'dwellings')
fig.savefig("dwellings_bar.png")
fig

In [ ]:
fig = plot_total_scenario_data(arc_dwellings, 'dwellings')
fig.savefig("dwellings_total.png")
fig

In [ ]:
fig = plot_scenario_data_highlight(arc_population, 'population')
fig.savefig("population_highlight.png")
fig

In [ ]:
fig = plot_total_scenario_data(arc_population, 'population')
fig.savefig("population_total.png")
fig

In [ ]:
fig = plot_scenario_data_highlight(arc_gva, 'gva')
fig.savefig("gva_highlight.png")
fig

In [ ]:
fig = plot_total_scenario_data(arc_gva, 'gva')
fig.savefig("gva_total.png")
fig

In [ ]:
fig = plot_total_scenario_data(arc_jobs, 'employment')
fig.savefig("employment_total.png")
fig

In [ ]:
fig = plot_scenario_data_highlight(arc_jobs, 'employment')
sbn.plt.title('Employment')
fig.savefig("employment_highlight.png")
fig

# Plot difference spatially

In [ ]:
sbn.choose_diverging_palette()

In [ ]:
def plot_difference_spatially(df, variable, log_values=False):
    df = df.copy()
    # pull out baseline values
    df_base = df[df.scenario == 'baseline'] \
        [['timestep', 'lad_uk_2016', variable]] \
        .rename(columns={variable: 'base'})
    # filter to drop baseline and base year
    df = df[(df.scenario != 'baseline') & (df.timestep > 2015)]
    # merge base baseline values to calculate diff
    diff_key = 'diff_{}'.format(variable)
    df = df.merge(df_base, how='left', validate='many_to_one', on=['timestep', 'lad_uk_2016'])
    if log_values:
        df[diff_key] = np.log(df[variable]) - np.log(df.base)
    else:
        df[diff_key] = df[variable] - df.base
    
    # set up colours
    cmap = sbn.diverging_palette(
        h_neg=256, 
        h_pos=12, 
        s=85, 
        l=57, 
        sep=1, 
        n=7, 
        as_cmap=True
    )
    vmax = df[diff_key].abs().max()

    def plot_diff_map(data, *args, **kwargs):
        cax = plt.gca()
        ax = data.plot(
            column=diff_key, 
            ax=cax, 
            cmap=cmap, 
            vmin=-vmax, 
            vmax=vmax
        )
        ax.set_axis_off()
        return ax

    with sbn.plotting_context('paper'):
        g = sbn.FacetGrid(
            data=df, 
            row='scenario', 
            col='timestep', 
            height=4, 
            sharex=True, 
            sharey=True
        )
        g.map_dataframe(plot_diff_map)

    return g

In [ ]:
fig = plot_difference_spatially(arc_dwellings, 'dwellings')
fig.savefig("dwellings_difference.png")
fig

In [ ]:
fig = plot_difference_spatially(arc_population, 'population')
fig.savefig('population_difference.png')
fig

In [ ]:
fig = plot_difference_spatially(arc_jobs, 'employment')
fig.savefig('employment_difference.png')
fig

In [ ]:
fig = plot_difference_spatially(arc_gva, 'gva')
fig.savefig('gva_difference.png')
fig

# Baseline Results

In [ ]:
results.available_results('arc_ets__baseline')

In [ ]:
results.list_sector_models('arc_ets__baseline')

In [ ]:
results.list_outputs('aggregate_energy_constrained')

In [ ]:
model_run_names = ['arc_ets__baseline', 'arc_ets__expansion']
model_names = ['aggregate_energy_constrained']
output_names = ['elecload']
timesteps=[2015, 2030, 2050]
elecload = arc_lads.join(
    results.read_results(
        model_run_names, model_names, output_names, timesteps).set_index('lad_uk_2016').drop(columns=['decision']))

In [ ]:
arc_elecload = elecload.reset_index().melt(
    id_vars=['index', 'geo_label', 'geometry', 'gid', 'timestep', 'hourly', 'model_run'], 
    value_vars='elecload')

In [ ]:
arc_elecload.head()

In [ ]:
data = arc_elecload.drop(columns=['geometry', 'gid'])
data = data.groupby(by=['model_run', 'timestep', 'hourly']).sum().reset_index()

sbn.relplot(x='hourly', y='value', row='model_run', col='timestep', data=data, kind='line')

# Visualise sos as a graph

In [ ]:
import networkx as nx

In [ ]:
def visualise_sos(store, model_run):
    """Write out a graphml file showing directed links between scenarios, models and adaptors
    """
    config = store.read_model_run(model_run)
    sos = store.read_sos_model(config['sos_model'])
    models = sos['sector_models']
    scenarios = sos['scenarios']
    nodes = models + scenarios
    G = nx.DiGraph(model_run='arc_ets__expansion', sos_model=sos['name'])
    for x in nodes:
        G.add_node(x, name=x) 
    for dep in sos['model_dependencies'] + sos['scenario_dependencies']:
        G.add_edge(dep['source'], dep['sink'])
    nx.write_graphml_lxml(G, "{}.graphml".format(model_run))

In [ ]:
store = results._store
model_run = 'arc_et__expansion'
visualise_sos(store, model_run)

# Debugging expansion scenario

In [ ]:
model_run = ['arc_ets__expansion']
model_names = ['energy_demand_constrained']
output_names = ['industry_gas_boiler_gas']
results.available_results(model_run[0])['sector_models'][model_names[0]]['outputs'].keys()

In [ ]:
data = results.read_results(model_run, model_names, output_names)

In [ ]:
# Show all entries where there is missing data
data[data['industry_gas_boiler_gas'].isnull()==True]

# Investigate GVA sectoral and total 

In [ ]:
def get_scenario_data(variable, timesteps):
    data = results.read_scenario_data('socio-economic', 'arc_baseline', variable, timesteps).rename(
        columns={variable: 'baseline'})

    data['unplanned'] = results.read_scenario_data(
        'socio-economic', 'arc_unplanned', variable, timesteps)[variable]
    data['expansion'] = results.read_scenario_data(
        'socio-economic', 'arc_expansion', variable, timesteps)[variable]
    data['new-cities'] = results.read_scenario_data(
        'socio-economic', 'arc_new-cities', variable, timesteps)[variable]
    
    return data

head = get_scenario_data('gva_per_head', [2015, 2030, 2050])
head.sample(5)

In [ ]:
gva = get_scenario_data('gva', [2015, 2020, 2030, 2050])
gva.sample(5)

In [ ]:
sector = get_scenario_data('gva_per_sector', [2015, 2020, 2030, 2050])

In [ ]:
sbn.relplot(data=sector.melt(id_vars=['lad_uk_2016', 'timestep', 'sectors'], 
                             var_name='scenario').groupby(by=['scenario', 'timestep']).sum().reset_index(), 
            x='timestep', y='value', 
            col='scenario', kind='line')

In [ ]:
gva_melted = gva.melt(id_vars=['lad_uk_2016', 'timestep'], var_name='scenario')

In [ ]:
summed = sector.melt(
    id_vars=['lad_uk_2016', 'timestep', 'sectors'], 
    var_name='scenario').groupby(
    by=['scenario', 'timestep']).sum().drop(columns='sectors')

In [ ]:
df = sector.melt(
    id_vars=['lad_uk_2016', 'timestep', 'sectors'], 
    var_name='scenario')
df[df['value'] == 0]

In [ ]:
gva_melted.groupby(by=['scenario', 'timestep']).sum()

# Results: Total emissions

In [ ]:
model_run_names = ['arc_ets__baseline']
model_names = ['energy_supply_constrained']
output_name = 'e_emissions_eh'
timesteps=[2015, 2020, 2030, 2050]

def get_total_results(model_run_names, model_names, output_name, timesteps):
    es_res = results.read_results(
            model_run_names, model_names, [output_name], timesteps)
    agg_results = es_res.drop(
        columns='decision').groupby(
        by=['model_run', 'timestep']).sum().reset_index()
    return agg_results[['model_run', 'timestep', output_name]].set_index(['model_run', 'timestep'])
    
e_emissions_eh = get_total_results(model_run_names, model_names, 'e_emissions_eh', timesteps)
e_emissions_bb = get_total_results(model_run_names, model_names, 'e_emissions', timesteps)
h_emissions_eh = get_total_results(model_run_names, model_names, 'h_emissions_eh', timesteps)

# fig = sbn.relplot(data=emissions_eh, row='model_run', x='timestep', y=output_name, kind='line')


In [ ]:
emissions = emissions_bb.join(emissions_eh).join(h_emissions_eh)
emissions['total'] = emissions['e_emissions'] + emissions['e_emissions_eh'] + emissions['h_emissions_eh']
data = emissions.reset_index().melt(id_vars=['model_run', 'timestep'])
fig = sbn.catplot(data=data, hue='variable', x='timestep', col='model_run', y='value', kind='bar')
fig.set_axis_labels('year', 'emissions (kgCO2)')

In [ ]:
results.list_outputs('energy_supply_constrained')

In [ ]:
import geopandas as gpd

def plot(df, spatial_dimension_name, output_name):

    spatial_dimension = store.read_dimension(spatial_dimension_name)['elements']
    spatial_dimension_features = [x['feature'] for x in spatial_dimension]

    spatial_df = gpd.GeoDataFrame.from_features(spatial_dimension_features)

    b = spatial_df.merge(df, left_on='name', right_on=spatial_dimension_name)

    b.plot(column=output_name)

In [ ]:
plot(elecload, 'energy_hub', 'e_emissions_eh')

In [ ]:
spatial_dimension = store.read_dimension('bus_bars')['elements']


In [ ]:
spatial_dimension_features = [x['feature'] for x in spatial_dimension]

In [ ]:
gpd.GeoDataFrame.from_features(spatial_dimension_features)

In [ ]:
spatial_df = gpd.GeoDataFrame.from_features(spatial_dimension_features)

b = spatial_df.merge(elecload, left_on='name', right_on='bus_bars')

In [ ]:
ax = b.plot(column='e_emissions')
arc_lads.plot(ax=ax, alpha=0.3)
ax.set_axis_off()

# Digital comms results

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame.from_csv('aggregate_scenario_metrics.csv')

# Transport for energy results

In [ ]:
results.available_results('arc_etmod-only__baseline')

In [ ]:
model_run_names = ['arc_etmod-only__baseline', 'arc_etmod-only__new-cities', 'arc_etmod-only__expansion']
model_names = ['et_module']
output_name = 'v2g_g2v_capacity'
timesteps=[2015, 2030, 2050]

df = get_total_results(model_run_names, model_names, output_name, timesteps)
df

In [ ]:
data = df.reset_index().melt(id_vars=['model_run', 'timestep'])
fig = sbn.catplot(data=data, hue='variable', x='timestep', col='model_run', y='value', kind='bar')

In [ ]:
data

In [ ]:
sbn.relplot(x='timestep', y='value', hue='model_run', data=data, kind='line')

In [ ]:
import pandas

In [ ]:
results.list_scenario_outputs('ev_transport_trips')
# ['electric_vehicle_electricity_consumption', 'electric_vehicle_trip_starts']
scenarios = ['arc_baseline', 'arc_new-cities', 'arc_expansion']
dfs = []
for s in scenarios:
    trips = results.read_scenario_data(
        'ev_transport_trips', s, 'electric_vehicle_electricity_consumption', [2015, 2030, 2050])
    trips['scenario'] = s
    dfs.append(trips)

trips = pandas.concat(dfs, axis=0)
trips.head()

In [ ]:
trips.tail()

In [ ]:
trips.annual_day_hours.unique()

In [ ]:
lu = {
 'MIDNIGHT': 0,
 'ONEAM': 1,
 'TWOAM': 2,
 'THREEAM': 3,
 'FOURAM': 4,
 'FIVEAM': 5,
 'SIXAM': 6,
 'SEVENAM': 7,
 'EIGHTAM': 8,
 'NINEAM': 9,
 'TENAM': 10,
 'ELEVENAM': 11,
 'NOON': 12,
 'ONEPM': 13,
 'TWOPM': 14,
 'THREEPM': 15,
 'FOURPM': 16,
 'FIVEPM': 17,
 'SIXPM': 18,
 'SEVENPM': 19,
 'EIGHTPM': 20,
 'NINEPM': 21,
 'TENPM': 22,
 'ELEVENPM': 23
}

In [ ]:
trips['hour'] = trips.annual_day_hours.apply(lambda h: lu[h])

In [ ]:
lad_codes = list(arc_lads.reset_index().geo_code)
trips_arc = trips[trips.lad_gb_2016.isin(lad_codes)]
trips_arc

In [ ]:
trips_total = trips_arc.groupby(
    ['timestep', 'annual_day_hours', 'hour', 'scenario']
).sum().reset_index().sort_values(
    ['scenario', 'timestep','hour']
)
trips_total

In [ ]:
sbn.relplot(
    x='hour', y='electric_vehicle_electricity_consumption', 
    hue='scenario',
    col='timestep',
    kind='line',
    estimator=None,
    data=trips_total)

In [ ]:
trips_by_lad = trips_arc.groupby(
    ['timestep', 'lad_gb_2016', 'scenario']
).sum()['electric_vehicle_electricity_consumption'].reset_index()
trips_by_lad

In [ ]:
# facet map plot

# melted_dwellings = reshape(df).drop(columns='geometry')

# def log(series):
#     return series.apply(lambda x: np.log(x))

# melted_dwellings['norm_value'] = log(melted_dwellings['value'])

baseline = trips_by_lad[trips_by_lad['scenario']=='arc_baseline'].drop(columns='scenario')
non_baseline = trips_by_lad[~(trips_by_lad['scenario']=='arc_baseline')]

diff = non_baseline.set_index(['scenario', 'lad_gb_2016', 'timestep']).sub(
            baseline.set_index(['lad_gb_2016', 'timestep'])).reset_index()
#diff = highlight(diff)  
diff

In [ ]:
diff_spatial = arc_lads.reset_index().set_index('geo_code').join(
                   diff.rename(columns={'lad_gb_2016':'geo_code'}).set_index('geo_code'), rsuffix='bla')
diff_spatial

In [ ]:
colname = 'electric_vehicle_electricity_consumption'
cmap = sbn.diverging_palette(h_neg=220, h_pos=10, s=74, l=50, sep=10, n=9, as_cmap=True)

def plot_diff_map(data, *args, **kwargs):
    cax = plt.gca()
    ax = data.plot(column=colname, ax=cax, cmap=cmap, 
                   vmin=-diff_spatial[colname].max(), vmax=diff_spatial[colname].max())
    ax.set_axis_off()
    return ax

# Remove baseline (no difference across scenarios)
diff_spatial = diff_spatial[~(diff_spatial['timestep'] == 2015)]

with sbn.plotting_context('paper'):
    g = sbn.FacetGrid(data=diff_spatial, row='scenario', col='timestep', 
                      height=4, sharex=True, sharey=True)
    g.map_dataframe(plot_diff_map)
